In [23]:
import requests

player_names = {}  # Dictionnaire global pour stocker les noms des joueurs déjà récupérés

def get_player_name(player_id: int) -> str:
    """
    Retrieves the name of a player given their ID.

    Args:
        player_id (int): The ID of the player to retrieve.
    
    Returns:
        str: The name of the player or 'Unknown Player' if not found.
    """
    if player_id is None:
        return None

    # Vérifier si le joueur est déjà dans le dictionnaire
    if str(player_id) in player_names:
        return player_names[str(player_id)]
    
    # Effectuer une requête à l'API pour récupérer les informations du joueur
    url = f"https://api-web.nhle.com/v1/player/{player_id}/landing"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lève une erreur si la requête échoue
        player_data = response.json()
        
        # Extraire les noms du joueur (en accédant à la clé 'default')
        first_name = player_data.get('firstName', {}).get('default', 'Unknown')
        last_name = player_data.get('lastName', {}).get('default', 'Player')
        full_name = f"{first_name} {last_name}"

        # Stocker dans le dictionnaire pour éviter des appels répétés
        player_names[str(player_id)] = full_name
        
        return full_name

    except requests.RequestException as e:
        print(f"Error fetching player data: {e}")
        return "Unknown Player"

 


def describe_event(event):
    # Extraire les informations de l'événement
    typeDescKey = event.get('typeDescKey', "")
    details = event.get('details', {})

    # Extraire les ID des joueurs s'ils sont présents
    losing_player_id = details.get('losingPlayerId')
    winning_player_id = details.get('winningPlayerId')
    give_away_id = details.get('playerId')
    blocking_player_id = details.get('blockingPlayerId')
    blocked_player_id = details.get('shootingPlayerId')
    shooting_player_id = details.get('shootingPlayerId')
    goalie_id = details.get('goalieInNetId')
    penalized_player_id = details.get('committedByPlayerId')
    penalty_player_id = details.get('drawnByPlayerId')
    hitting_player_id = details.get('hittingPlayerId')
    hit_player_id = details.get('hitteePlayerId')

    # Récupérer les noms des joueurs si les IDs sont présents
    losing_player = get_player_name(losing_player_id) if losing_player_id else None
    winning_player = get_player_name(winning_player_id) if winning_player_id else None
    shooting_player = get_player_name(shooting_player_id) if shooting_player_id else None
    goalie = get_player_name(goalie_id) if goalie_id else None
    penalized_player = get_player_name(penalized_player_id) if penalized_player_id else None
    penalty_player = get_player_name(penalty_player_id) if penalty_player_id else None
    hitting_player = get_player_name(hitting_player_id) if hitting_player_id else None
    hit_player = get_player_name(hit_player_id) if hit_player_id else None
    give_away = get_player_name(give_away_id) if give_away_id else None

    # Créer une description basée sur le type d'événement
    if typeDescKey == "faceoff" and winning_player and losing_player:
        return f"{winning_player} won a faceoff against {losing_player}."
    
    if typeDescKey == "goal" and shooting_player and goalie:
        return f"{shooting_player} scored a goal against {goalie}."

    if typeDescKey == "shot-on-goal" and shooting_player and goalie:
        return f"{shooting_player} took a shot on goal saved by {goalie}."

    if typeDescKey == "giveaway" and give_away_id:
        player = get_player_name(give_away_id)
        return f"{player} gave the puck away."

    if typeDescKey == "period-start":
        return "The period has started."

    if typeDescKey == "blocked-shot" and blocking_player_id and blocked_player_id:
        blocking_player = get_player_name(blocking_player_id)
        blocked_player = get_player_name(blocked_player_id)
        return f"{blocking_player} blocked a shot from {blocked_player}."

    if typeDescKey == "delayed-penalty":
        return "A delayed penalty was called."

    if typeDescKey == "failed-shot-attempt" and shooting_player:
        return f"{shooting_player} attempted a shot but missed the net."

    if typeDescKey == "shootout-complete":
        return "The shootout has ended."

    if typeDescKey == "stoppage":
        return "The game was stopped."

    if typeDescKey == "penalty" and penalized_player and penalty_player:
        return f"Penalty: {penalized_player} committed a foul against {penalty_player}."

    if typeDescKey == "takeaway" and give_away:
        return f"{give_away} took the puck away."

    if typeDescKey == "missed-shot" and shooting_player:
        return f"{shooting_player} missed the net with a shot."

    if typeDescKey == "game-end":
        return "The game has ended."

    if typeDescKey == "hit" and hitting_player or hit_player:
        return f"{hitting_player} delivered a hit on {hit_player}."

    if typeDescKey == "period-end":
        return "The period has ended."

    # Par défaut, retourner une chaîne vide ou un message de type non supporté
    return "Event description not available."





In [31]:
import json
import os
import ipywidgets as widgets
from IPython.display import clear_output, display, HTML
import base64
from PIL import Image as PILImage, ImageDraw
import numpy as np


def playerId_type(event):
    event_type = event.get('typeDescKey', 'N/A')
    details = event.get('details', {})
    
    if event_type == 'hit':
        return {
            "player_1": {"full name": get_player_name(details.get('hittingPlayerId'))},
            "player_2": {"full name": get_player_name(details.get('hitteePlayerId'))}
        }
    elif event_type == 'faceoff':
        return {
            "player_1": {"full name": get_player_name(details.get('winningPlayerId'))},
            "player_2": {"full name": get_player_name(details.get('losingPlayerId'))}
        }
    elif event_type == 'blocked-shot':
        return {
            "player_1": {"full name": get_player_name(details.get('blockingPlayerId'))},
            "player_2": {"full name": get_player_name(details.get('shootingPlayerId'))}
        }
    elif event_type in ['missed-shot', 'shot-on-goal']:
        return {
            "player_1": {"full name": get_player_name(details.get('shootingPlayerId'))},
            "player_2": {"full name": get_player_name(details.get('goalieInNetId'))}
        }
    elif event_type == 'giveaway':
        return {
            "player_1": {"full name": get_player_name(details.get('playerId'))}
        }
    else:
        return {}


    
class NHLDataParser:
    def __init__(self, nhl_games_file_path, season=None, stage='regular'):
        self.season = season
        self.stage = stage
        self.game_type = {'regular': 2, 'playoff': 3}

        if os.path.isdir(nhl_games_file_path):
            json_file_path = os.path.join(nhl_games_file_path, 'all_games_data.json')
        else:
            json_file_path = nhl_games_file_path

        with open(json_file_path, 'r') as file:
            self.games_data = json.load(file)

        self.filtered_games = self.filter_games_by_season_and_stage()

    def filter_games_by_season_and_stage(self):
        filtered = {}
        for game_id, game_data in self.games_data.items():
            if (self.season is None or game_data['season'] == int(self.season)) and game_data['gameType'] == self.game_type[self.stage]:
                filtered[game_id] = game_data
        return filtered

    def get_game_ids(self):
        return list(self.filtered_games.keys())

    def get_events(self, game_id):
        return self.filtered_games[game_id]['plays'] if game_id in self.filtered_games else []
    
    

    def get_game_info(self, game_id):
        """Retourne les informations du match (Game ID, équipes, score, etc.)."""
        game_data = self.filtered_games.get(game_id)
        if not game_data:
            return "Données du match indisponibles"
        
        home_team = game_data['homeTeam']['name']['default']
        away_team = game_data['awayTeam']['name']['default']
        home_score = game_data['homeTeam']['score']
        away_score = game_data['awayTeam']['score']
        game_starting_time = game_data['startTimeUTC']
        home_sog = game_data['homeTeam']['sog']
        away_sog = game_data['awayTeam']['sog']

        formatted_info = f"""
        <p><b>{game_starting_time}</b></p>
        <p><b>Game ID: {game_id}; {home_team} (home) vs {away_team} (away)</b></p>
        <p><b>{game_data['periodDescriptor']['periodType']}</b></p>
        <table style="width:50%; text-align: left; border-collapse: collapse;">
           <tr>
               <th></th>
               <th style="text-align: center;">Home</th>
               <th style="text-align: center;">Away</th>
           </tr>
           <tr>
               <td><b>Teams:</b></td>
               <td style="text-align: center;">{home_team}</td>
               <td style="text-align: center;">{away_team}</td>
           </tr>
           <tr>
               <td><b>Goals:</b></td>
               <td style="text-align: center;">{home_score}</td>
               <td style="text-align: center;">{away_score}</td>
           </tr>
           <tr>
               <td><b>SoG:</b></td>
               <td style="text-align: center;">{home_sog}</td>
               <td style="text-align: center;">{away_sog}</td>
           </tr>
       </table>
       """
        return formatted_info

    def get_event_coordinates(self, event):
        """Retourne les coordonnées x et y d'un événement."""
        return event.get('details', {}).get('xCoord'), event.get('details', {}).get('yCoord')


# Fonction pour afficher la patinoire avec la position du joueur marquée
def show_rink_with_player_position(x, y, rink_display_label,event):
    # Charger l'image de la patinoire
    img_path = 'C:\\Users\\Admin\\Documents\\IFT6758-A3\\IFT-6758---A3-NHL\\notebooks\\NHL_image.PNG'

    if x is not None and y is not None:
        # Ouvrir l'image
        img = PILImage.open(img_path)
        draw = ImageDraw.Draw(img)

        # Taille de la patinoire et de l'image
        rink_origin_x, rink_origin_y = 112, 80  # Coordonnées du coin inférieur gauche de la patinoire
        rink_width, rink_height = 917, 389      # Taille de la patinoire
        img_width, img_height = img.size

        point_size = 10  # Taille du point à dessiner

        # Convertir les coordonnées du joueur pour qu'elles correspondent à la patinoire
        x_rink = rink_width / 200 * (x + 100)  # Conversion des coordonnées de -100 à 100 en pixels
        y_rink = rink_height / 85 * (42.5 - y)*0.95 # Conversion des coordonnées de -42.5 à 42.5 en pixels
        
        # Ajouter les offsets de la patinoire dans l'image
        x_img = rink_origin_x + x_rink
        y_img = rink_origin_y + y_rink

        # Dessiner un point à la position (x_img, y_img)
        # Dessiner un point à la position (x_img, y_img)
        if event.get('typeDescKey', 'N/A') == 'missed-shot':
            # Point rouge barré pour un tir manqué
            draw.ellipse([(x_img - point_size, y_img - point_size), 
                          (x_img + point_size, y_img + point_size)], outline='red', width=2)
            draw.line([(x_img - point_size, y_img - point_size), 
                       (x_img + point_size, y_img + point_size)], fill='red', width=2)
        else:
            # Point bleu pour les autres événements
            draw.ellipse([(x_img - point_size, y_img - point_size), 
                          (x_img + point_size, y_img + point_size)], fill='blue')

        # Sauvegarder et afficher l'image temporairement
        temp_image_path = "temp_rink_with_point.png"
        img.save(temp_image_path)

        # Ouvrir l'image en base64 pour l'afficher dans un widget HTML
        with open(temp_image_path, "rb") as img_file:
            b64_string = base64.b64encode(img_file.read()).decode('utf-8')

        # Afficher l'image dans un widget HTML
        rink_display_label.clear_output()  # Vider avant d'afficher une nouvelle image
        with rink_display_label:
            display(HTML(f'<img src="data:image/png;base64,{b64_string}" style="width: 50%; height: auto;" />'))
    else:
        rink_display_label.clear_output()  # Pas de coordonnées pour cet événement



# Initialisation du parser
nhl_games_file_path = 'C:\\Users\\Admin\\Documents\\IFT6758-A3\\IFT-6758---A3-NHL\\nhl_data'
parser = NHLDataParser(nhl_games_file_path, season='20162017', stage='regular')

# Widgets pour l'interface utilisateur
game_ids = parser.get_game_ids()
game_id_slider = widgets.IntSlider(
    value=int(game_ids[0]),
    min=int(game_ids[0]),
    max=int(game_ids[-1]),
    step=1,
    description='Game ID:',
    continuous_update=False
)

event_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=0,
    step=1,
    description='Event:',
    continuous_update=False
)

game_info_label = widgets.Output()
event_info_label = widgets.Output()
rink_display_label = widgets.Output()
even_description_label = widgets.Output(layout=widgets.Layout(display='flex', margin = '0 100px'))


# Fonction pour mettre à jour les événements en fonction du `game_id` sélectionné
def update_event_slider(*args):
    game_id = str(game_id_slider.value)
    events = parser.get_events(game_id)
    event_slider.max = len(events) - 1 if events else 0

    # Mettre à jour les informations du match
    with game_info_label:
        game_info_label.clear_output()
        display(HTML(parser.get_game_info(game_id)))

    # Mettre à jour le slider des événements
    event_slider.value = 0  # Réinitialiser à 0 chaque fois qu'on change de match




def format_dict(d, indent=0):
    result = ""
    for key, value in d.items():
        # Vérifier si la clé est l'une des sections spéciales
        if key in ['About', 'Results', 'Players']:
            result += '  ' * indent + f"<b>{key}:</b> "  # Ajoute les balises <b> autour du titre
        else:
            result += '  ' * indent + f"{key}: "
        
        if isinstance(value, dict):
            result += "\n{\n" + format_dict(value, indent + 1) + '  ' * indent + "}\n"
        else:
            result += f"{value}\n"
    return result


# Fonction pour afficher l'événement sélectionné
def display_event(game_id, event_idx):
    clear_output(wait=True)
    display(vbox)

    events = parser.get_events(str(game_id))
    if events and 0 <= event_idx < len(events):
        event = events[event_idx]
        
        # Récupérer la description de l'événement
        event_description = describe_event(event)

    
        
        # Afficher les informations de l'événement
        # Créer le dictionnaire des informations de l'événement
        event_info = {
             "About": {
        "eventId": event.get('eventId', 'N/A'),
        "periodType": event.get('periodDescriptor', {}).get('periodType', 'N/A'),
        "maxRegulationPeriods": event.get('periodDescriptor', {}).get('maxRegulationPeriods', 'N/A'),
        
        "timeInPeriod": event.get('timeInPeriod', 'N/A'),
        "timeRemaining": event.get('timeRemaining', 'N/A'),
        "situationCode": event.get('situationCode', 'N/A'),
        "homeTeamDefendingSide": event.get('homeTeamDefendingSide', 'N/A'),
        "typeCode": event.get('typeCode', 'N/A'),
        
        "sortOrder": event.get('sortOrder', 'N/A'),
        "coordinates": {"x": event.get('details', {}).get('xCoord', 'N/A'),"y": event.get('details', {}).get('yCoord', 'N/A') }
           },
          "Players":{
        "Players": playerId_type(event)
             },
        "Results":{"description":describe_event(event),
                   "event": event.get('typeDescKey', 'N/A'),
                   }

        }
        


        formatted_info = format_dict(event_info)
        
        # Afficher l'image de la patinoire avec les coordonnées de l'événement
        x_coord, y_coord = parser.get_event_coordinates(event)

        # Afficher l'image de la patinoire avec le joueur et le titre comme description de l'événement
        show_rink_with_player_position(x_coord, y_coord, rink_display_label,event)


        # Mettre à jour le even_description_label avec la description de l'événement
        with even_description_label:
            even_description_label.clear_output()
            display(HTML(f'<h4>{event_description}</h4>'))  # Affichage de la description
        
        # Afficher la description de l'événement comme titre
        with event_info_label:
            event_info_label.clear_output()
            display(HTML(f'<pre style="font-size:10px;">{formatted_info}</pre>'))

    else:
        with event_info_label:
            event_info_label.clear_output()
            display("No event data available for this game or invalid event index.")

# Associer les changements de sliders aux événements
game_id_slider.observe(update_event_slider, 'value')
event_slider.observe(lambda change: display_event(game_id_slider.value, event_slider.value), 'value')


# Conteneur pour afficher les widgets
vbox = widgets.VBox([game_id_slider, game_info_label, event_slider,even_description_label, rink_display_label,event_info_label])
display(vbox)

# Initialiser l'affichage
update_event_slider()
